**Metaclasses**

The class that defines the creation of a class. Class definitions create a class name, a class dictionary, and a list of base classes. The metaclass is responsible for taking those three arguments and creating the class. They let you intercept the class statement and provide special behaviour each time a class is defined. The default metaclass is 'type':

In [12]:
MyClass = type('Foo', (object,), {'foo': 'bar'})
a = MyClass()
a.foo

'bar'

A metaclass is defined by inheriting form type. In the default case, a metaclass receives the contents of associated class statetments in its \__new\__ method. Here you can modify the class information before the type is actually constructed. The metaclass has access to the name of the class, parents it inherits form, and all of the class attributes defined in the body:

In [54]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(meta, name, bases, class_dict)
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass

c = MyClass()

<class '__main__.Meta'> MyClass () {'foo': <function MyClass.foo at 0x0000016802A8E400>, '__qualname__': 'MyClass', '__module__': '__main__', 'stuff': 123}


A simple application of metaclasses is verifying that a class was defined correctly when you are building a complex class hierarchy. The \__new\__ method of metaclasses is run after the class statement’s entire body has been processed.   Often a class's validation code runs in the \__init\__ method; using metaclasses for validation can raise errors much earlier. 

Validation of a class's parameter can be achieved by adding functionality to the Meta.\__new\__ method. In the following example, the abstract Polygon class (whose base class is object) is not validated, but all classes which inherit from it are:

In [64]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        if bases !=	(object,):
            if class_dict["sides"] < 3:
                raise ValueError("Polygons need 3+ sides")
        return type.__new__(meta, name, bases, class_dict)

class Polygon(object, metaclass=ValidatePolygon):
    sides = None
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides -2) * 90
    
class Triangle(Polygon):
    sides = 3
    
Triangle.interior_angles()

90

**Method resolution order**
When you have a hierarchy of classes, how does Python decide which method to use when  MyClass.\__mro\__ will gives a tuple showing the order in which methods and attributes will be looked up:

In [13]:
class Animal(object):
    pass

class Dog(Animal):
    pass

class Labrador(Dog):
    pass

Labrador.__mro__

(__main__.Labrador, __main__.Dog, __main__.Animal, object)

The MRO in action:

In [14]:
class A(object):
    def foo(self):
        print('In A')

class B(A):
    pass

class C(A):
    def foo(self):
        print('In C')
        
class D(B, C):
    pass

d = D()
d.foo()

In C


**Multiprocessing and multithreading**

Concurrency is when two or more tasks can start, run, and complete in overlapping time periods. It doesn't necessarily mean they'll ever both be running at the same instant. Eg. multitasking on a single-core machine. Parallelism is when two or more tasks  run at the same time, e.g. on a multicore processor. Concurrency can be characterized as a property of a program or system and parallelism as the run-time behaviour of executing multiple tasks at the same time. 

![](images/concurrency_parallelism.png)
If we ran this program on a computer with a single CPU core, the OS would be switching between the two threads, allowing one thread to run at a time. If we ran this program on a computer with a multi-core CPU then we would be able to run the two threads in parallel - side by side at the exact same time

A thread is a sequence of instructions within a process. It can be thought of as a lightweight process. Threads share the same memory space. A process is an instance of a program running in a computer which can contain one or more threads. A process has its independant memory space. 
    
The threading module is used for working with threads in Python. The CPython implementation has a Global Interpreter Lock (GIL) which allows only one thread to be active in the interpreter at once. The mechanism used by the CPython interpreter to assure that only one thread executes Python bytecode at a time. This simplifies the CPython implementation by making the object model (including critical built-in types such as dict) implicitly safe against concurrent access. Locking the entire interpreter makes it easier for the interpreter to be multi-threaded, at the expense of much of the parallelism afforded by multi-processor machines. This means that threads cannot be used for parallel execution of Python code. While parallel CPU computation is not possible, parallel IO operations are possible using threads. This is because performing IO operations releases the GIL. What are threads used for in Python? 

    In GUI applications to keep the UI thread responsive

    IO tasks (network IO or filesystem IO)

Using threads for these tasks improves performance, since in network IO for example, most of the time is spent waiting for a response from the URL. Threads should not be used for CPU bound tasks as this will actually result in worse performance compared to using a single thread.

For parallel execution of tasks use multiprocessing, a package that supports spawning processes using an API similar to the threading module. It side-steps the GIL by using subprocesses instead of threads. The Pool object which offers a convenient means of parallelizing the execution of a function across multiple input values, distributing the input data across processes (data parallelism).

**Mutable default arguments**

In the following example, the list object has an empty list as a default value. This assignment is carried out only when the function definition is first evaluated. 

In [28]:
def func1(item, lst=[]):
    lst.append(item)
    return lst

def func2 (item, lst=None):
    if lst is None: 
        lst = []
    lst.append(item)
    return lst

A new list is created each time the function is called if a second argument isn’t provided, so that the output is as follows:

In [29]:
print([func1(i) for i in range(3)])

[[0, 1, 2], [0, 1, 2], [0, 1, 2]]


This feature gives speed and memory boosts, as in most cases you will have immutable default arguments and Python can construct them just once, instead of on every function call. Another benefit is simplicity, as it is easier to understand how the expression is evaluated and thereby make debugging easier. 

To avoid this, the mutable objects used as defaults should be replaced by None, and then the arguments tested for None:

In [30]:
print([func2(i) for i in range(3)])

[[0], [1], [2]]


**Polymorphism**

Refers to the ability of an object to provide different behaviors (use different implementations) depending on its own nature. Specifically, depending on its position in the class hierarchy. Polymorphism is declaring a uniform interface that isn't type aware, leaving implementation details to concrete types that implement the interface. It allows the expression of some sort of contract, with potentially many types implementing that contract (whether through class inheritance or not) in different ways, each according to their own purpose. Code using that contract should not(*) have to care about which implementation is involved, only that the contract will be obeyed.

Polymorphism can be achieved by *method overriding*,  when a method defined in a superclass or interface is re-defined by one of its subclasses, thus modifying/replacing the behavior the superclass provides. Notice the signature of the method remains the same when overriding. *Method overloading* is unrelated to polymorphism. It refers to defining different forms of a method. In this case the signature of the method is changed. 

**Property decorator and the uniform access principle** 

Python code strives to adhere to the Uniform Access Principle; the are no truly 'protected' or 'private' attributes. Getters and setters are used in many object oriented programming languages to ensure the principle of data encapsulation (the bundling of data with the methods that operate on these data). The accepted approach in Python is to xxpose your instance variables directly, e.g. foo.x = 0, not foo.set_x(0), which preserves the access semantics. The main advantage to this approach is that the caller gets to do this: foo.x += 1 instead of the less-readable: foo.set_x(foo.get_x() + 1)

Instance variables starting with a single underscore are conventionally private; not to be messed with directly. and they shouldn't mess with it directly. With double underscore, Python mangles the name but the variable is still accessible from outside.

If you need to wrap the access variables assigned by methods use @property. Getter, setter and deleter methods enable you to set an attribute using a function. Getting access is the same and the setter method allows you to have functionality of normal class attribute assignment. You can start with the simplest implementation imaginable, and you are free to later migrate to a version which preserves the access semantics and so avoids having to change the interface.

Specifying a `setter` on a property also lets you perform type checking and validation on values passed to the class. See the 'Validation of class attributes' section below for the difference between property decorators and descriptors for attribute validation.

In [15]:
class A(object):
    _x = 0
    '''A._x is an attribute'''

    @property
    def x(self):
        '''
        A.x is a property
        This is the getter method
        '''
        return self._x

    @x.setter
    def x(self, value):
        """
        This is the setter method
        where I can check it's not assigned a value < 0
        """
        if value < 0:
            raise ValueError("Must be >= 0")
        self._x = value

a = A()
a._x = -1 
a.x = 2
# raises ValueError:
# a.x = -1 

In [32]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0   

        
class VoltageResistor(Resistor):
    """ Migrate to property decorator to allow current to be varied by assigning the voltage property """
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property 
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
        
class FixedResistor(Resistor):
    """ Use @property to make attribtues from parent classes immutable """
    def __init__(self, ohms):
        super().__init__(ohms)
        self._ohms = ohms
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, 'ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms
        
r1 = Resistor(500)
r2 = VoltageResistor(500)
r3 = FixedResistor(500)

# Immutable, would raise AttributeError:
# r3.ohms = 5
r2.voltage = 12
r2.current

0.024

**Recursion**

A function is recursive if it calls itself and has a termination condition. Why a termination condition? To stop the function from calling itself ad infinity. An example of recursion in English: “A human is someone whose mother is human”. Also, a tree diagram where each branch is like a new tree.

The two key elements of a recursive algorithm are:

The termination condition: n == 0
The reduction step where the function calls itself with a smaller number each time: factorial(n - 1)

In [2]:
import logging

def factorial(n):
    if n == 0:
        return 1
    else:
        logging.warning(n)
        return n * factorial(n - 1)

factorial(3)

6

**Strong-typing** You cannot coerce objects into a different type by any kind of inferring, as in weakly-typed languages. As so, the object type is more explicit.



**`super()`**

`super()` calls only one class `__init__`

In [36]:
class A:
    def __init__(self):
        print("A")

class B:
    def __init__(self):
        print("B")

class C(B, A):
    def __init__(self):
        super().__init__() 

c=C() # n.b. is defined as class C(A, B) it would reach class A first and print "A"

B


Defining `super().__init__()` in all the classes will resolve the problem. In the following example, the `__init__` method of A is called first and then the `super()` function in A is called so it goes to class B and executes both print functions before printing "A":

In [38]:
class A:
    def __init__(self):
        print("reached A")
        super().__init__()
        print("A")

class B:
    def __init__(self):
        print("reached B")
        super().__init__()
        print("B")

class C(A,B):
    def __init__(self):
        super().__init__() 

c=C()

reached A
reached B
B
A


**Validation of class attributes** 

This can be done using property decorators or with descriptors

In [43]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length #if self._lenght is used, then it will not validate through setter. 
        self.width = width

    @property
    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * (self.length + self.width)

    @property
    def length(self):
        return self._length

    @length.setter
    def length(self, value):
        if not isinstance(value, int): #validating length
            raise TypeError("Only integers are allowed")
        self._length = value

r = Rectangle(3,2)
r.length = 4
r.area

8

We may also want to provide validation for multiple attributes (e.g. width as well as length). Descriptors would help  useful in this case:

In [51]:
class Integer: 
    def __init__(self, parameter):
        self.parameter = parameter

    def __get__(self, instance, owner):
        if instance is None: 
            return self
        else: 
            return instance.__dict__[self.parameter]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError("Interger value is expected")
        instance.__dict__[self.parameter] = value

class Rect: 
    length = Integer('length')
    width =  Integer('width')

    def __init__(self, length, width):
        self.length = length
        self.width = width

r = Rect(2, 1)
r.length

2